In [ ]:
import itertools
import math
import numpy as np

In [ ]:
%run -i utils.py

### Parameters definition

In [ ]:
# Number of prosumer
n_players = 4
# Seller price fixed
pv = 2
# Buyer price fixed
pa = 1
# Energy for each prosumer
#energy = np.array([335, -495, -96, 541], dtype = float) #simulation
#energy = np.array([-798, -825, -757, -649], dtype = float) #winter
energy = np.array([-335, -495, -96, 541], dtype = float) # summer

# Resistance of the distribution lines 
R = 0.2
# Coefficient that quantify the power lost in the transformer at the M-G
beta = 0.02
# Mean value of the voltage
U0 = 50
U1 = 22

# Distances from macro grid
d = np.array([4.95, 5.09, 5.87, 5.87])
# Matrix distances between prosumers
distance = np.array([[0,0.2,10.82,10.82],[0.2,0,10.97,10.96],[10.82,10.97,0,0.14],[10.82,10.96,0.14,0]])

### Simulation

In [ ]:
players = np.arange(1, n_players+1)

payoff_list = {
    (0,): 0
}
for L in range(1, len(players)+1):
    for subset in itertools.combinations(players, L):
        v_2 = [0]
        subset = np.array(subset)
        Q = energy[subset-1]
        
        u_sell = [0]*len(Q[Q>0])
        u = [0]*len(Q[Q<0])
        
        psm = price(subset, Q, pv)
        
        if len(subset)>1:
            for h in range(len(payoff_list)):
                if len(list(payoff_list.keys())[h]) < len(subset):
                    for k in range(len(list(payoff_list.keys())[h])):
                        if any(s == list(list(payoff_list.keys())[h])[k] for s in subset):
                            v_2.append(list(payoff_list.values())[h])
                            break
            
        val = payoff(subset, Q, energy, psm, u_sell, u)
        payoff_list[tuple(subset)] = val
        v_2.append(val)
        print('Value function: ', np.round(val))
        shap = np.array(shapley_value(len(Q),v_2))[:,1]
        
        for i in range(len(subset)):
            print('Player: ',subset[i], 'Shapley value: ', np.round(shap[i].astype(np.float)))